# Initiailize VW executor

In [ ]:
from vw_executor.vw import Vw

vw = Vw('.vw_cache')

# Get the data

In [ ]:
inputs = [
    'vw_executor/tests/data/cb_100_0.json',
    'vw_executor/tests/data/cb_100_1.json', 
    ]
inputs

# Define options grid and train

In [ ]:
from vw_executor.vw_opts import Grid
import pandas as pd

opts = Grid({
    '#base': ['--cb_explore_adf -P 10000 --preserve_performance_counters --save_resume --dsjson'],
    '--cb_type': ['ips', 'mtr']})
opts

In [ ]:
result = vw.train(inputs, opts)
result

## Get access to losses and job artifacts

In [ ]:
print(f'loss[0]: {result[0].loss}')
print(f'outputs: {result[0].outputs}')

# Pandas wrappers

In [ ]:
opts = pd.DataFrame(Grid({
    '#base': ['--ccb_explore_adf -P 10 --preserve_performance_counters --save_resume'],
    '--cb_type': ['ips', 'mtr'],
    '#format': ['', '--dsjson']}))
opts

In [ ]:
result = vw.train(inputs, opts)
result

## Investigate failure

In [ ]:
result[result['!Status'] == 'Failed'].iloc[0]['!Job'].failed.stdout.raw

## Explore metrics 

In [ ]:
result[result['!Status'] == 'Success'].iloc[0]['!Job'].loss_table

In [ ]:
import matplotlib.pyplot as plt
[(-r['!Job'].loss_table['loss']).plot(label=f"{r['!Job'].name}", figsize=(10,6)) for i, r in result[result['!Status'] == 'Success'].iterrows()]
plt.legend()

# Interactive mode

In [ ]:
%matplotlib widget 
from vw_executor.vw_opts import InteractiveGrid as Grid
opts = Grid({
    '#base': ['--cb_explore_adf -P 10 --preserve_performance_counters --save_resume --dsjson'],
    '--cb_type': ['ips', 'mtr'],
    '--learning_rate': [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]})
vw.train(inputs, opts)

# Get more logging

## Log to console

In [ ]:
from vw_executor.vw import Vw
from vw_executor.loggers import ConsoleLogger

vw = Vw('.vw_cache',
    loggers = [ConsoleLogger('DEBUG')])

vw.train(inputs, '--cb_explore_adf --dsjson')

## Log to file

In [ ]:
from vw_executor.vw import Vw
from vw_executor.loggers import FileLogger

vw = Vw('.vw_cache',
    loggers = [FileLogger('log.txt', 'DEBUG')])

vw.train(inputs, '--cb_explore_adf --dsjson')

## Log to multiple files

In [ ]:
from vw_executor.vw import Vw
from vw_executor.loggers import FileLogger

vw = Vw('.vw_cache',
    loggers = [MultiFileLogger('artifacts', 'DEBUG')])

vw.train(inputs, ['--cb_explore_adf --dsjson --epsilon 0.2', '--cb_explore_adf --dsjson --epsilon 0.3'])

## Publish artifacts

In [ ]:
from vw_executor.vw import Vw
from vw_executor.handlers import ArtifactCopy

vw = Vw('.vw_cache',
    handlers = [ArtifactCopy('artifacts', outputs=['-p'])])

vw.train(inputs, ['--cb_explore_adf --dsjson --epsilon 0.2', '--cb_explore_adf --dsjson --epsilon 0.3'], ['-p'])